Исследование поведения пользователей 2018

    - Получите данные из файла.
    - Произведите обзор данных и преобразование данных, если оно необходимо.

In [47]:
import pandas as pd
import numpy as np

purchase = pd.read_csv(r'C:\Users\Yana\Documents\purchase.csv', sep=',')
events = pd.read_csv(r'C:\Users\Yana\Documents\7_4_Events.csv', sep=',')

In [48]:
cond2018 = (events.start_time>='2018-01-01') & (events.start_time<'2019-01-01') & (events.event_type=='registration')
registered = events[cond2018]  
lst_reg = registered['user_id'].to_list() # список пользователей, зарег. в 2018
# отфильтровать пользователей зарег в 2018
events_df = events[events.user_id.isin(lst_reg)].copy() 
purchase_df = purchase[purchase.user_id.isin(lst_reg)].copy() 

In [49]:
purchase_df['event_type'] = 'purchase' # добавить ивент тип покупка
events_df = events_df.rename(columns={"id": "event_id"}) # переименовать айди
purchase_df = purchase_df.rename(columns={"id": "purchase_id"}) # переименовать айди
purchase_df['event_datetime'] = pd.to_datetime(purchase_df['event_datetime'])
total_events_df = pd.concat([events_df,purchase_df],sort=False) # объединить датафреймы

In [50]:
# изменить тип объект на дата для колонок с датой
total_events_df['start_time'] = pd.to_datetime(total_events_df['start_time'])
total_events_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 68559 entries, 51405 to 2778
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   event_id        66959 non-null  float64       
 1   event_type      68559 non-null  object        
 2   selected_level  8342 non-null   object        
 3   start_time      66959 non-null  datetime64[ns]
 4   tutorial_id     32954 non-null  float64       
 5   user_id         68559 non-null  int64         
 6   purchase_id     1600 non-null   float64       
 7   event_datetime  1600 non-null   datetime64[ns]
 8   amount          1600 non-null   float64       
dtypes: datetime64[ns](2), float64(4), int64(1), object(2)
memory usage: 5.2+ MB


Есть ли зависимость между выбранным уровнем сложности и вероятностью оплаты?

In [78]:
user_levels = total_events_df[~total_events_df['selected_level'].isnull()].copy()
user_levels_df = user_levels[['selected_level', 'user_id', 'start_time']]
print(user_levels_df['user_id'].nunique()) #8342 юзеров выбрали уровень сложности
print(user_levels_df.groupby('user_id')['selected_level'].count().mean()) #один уровень на юзера

8342
1.0


medium    4645
easy      2448
hard      1249
Name: selected_level, dtype: int64

In [81]:
user_levels_df['selected_level'].value_counts()
# сколько пользователей в каждой группе по уровню сложности

medium    4645
easy      2448
hard      1249
Name: selected_level, dtype: int64

In [80]:
merged_df_1 = user_levels_df.merge(purchase_df, on='user_id', how='outer')
# merged_df_1.groupby('selected_level')['purchase_id'].count() 
merged_df_1.pivot_table(values = ['purchase_id'], index = 'selected_level', aggfunc='count')
# сколько пользователей из каждой группы совершили покупку

,purchase_id
selected_level,
easy,189
hard,442
medium,969


In [89]:
merged_buyers_only = merged_df_1[~merged_df_1['purchase_id'].isnull()].copy()
merged_buyers_only['timedelta'] = merged_buyers_only['event_datetime'] - merged_buyers_only['start_time']

display(merged_buyers_only.groupby('selected_level')['timedelta'].mean())
merged_buyers_only.groupby('selected_level')['timedelta'].quantile([.25, .5, .75])

selected_level
easy     3 days 14:58:52.941798941
hard     3 days 07:20:41.420814479
medium   3 days 23:14:13.165118679
Name: timedelta, dtype: timedelta64[ns]

selected_level      
easy            0.25          1 days 17:18:56
                0.50          3 days 06:03:50
                0.75          5 days 06:58:18
hard            0.25   1 days 14:57:23.500000
                0.50   3 days 03:13:57.500000
                0.75   4 days 19:16:00.250000
medium          0.25          2 days 01:20:07
                0.50          3 days 19:53:19
                0.75          5 days 16:07:19
Name: timedelta, dtype: timedelta64[ns]

    - Выделите группы пользователей (по аналогии с группами, которые мы выделяли, когда находили зависимость между фактом прохождения обучения и оплатой).
    - Рассчитайте для каждой группы процент оплат (это мы тоже уже делали).
    - Для каждой группы сделайте по два датафрейма: один — с событиями выбора уровня сложности, другой — с событиями оплаты.
    - Объедините датафреймы в рамках одной группы и найдите разницу во времени между событиями (по аналогии с расчётом, который мы уже производили, когда находили время между событиями для всех пользователей).
    - Рассчитайте среднее время между событиями.